# This notebook converts the sqlite3 db to heroku postgresql

## Add your own postgresql uri

deployment should look like this:<br>
`deployment = ["sqlite:///<Your DB>.sqlite3",
                 "postgresql://<user>:<password>@<host>:<port>/<database>"]`


In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from secret import deployment
import secrets
import uuid

In [8]:
app_psql = Flask(__name__)
app_psql.config['SQLALCHEMY_DATABASE_URI'] = deployment[1]
app_psql.config['SECRET_KEY'] = secrets.token_hex(15)
app_psql.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True
db_psql = SQLAlchemy(app_psql)


class user_db(db_psql.Model):
    id = db_psql.Column(db_psql.String(200), primary_key=True)
    admin = db_psql.Column(db_psql.Boolean)
    name = db_psql.Column(db_psql.String(200))
    email = db_psql.Column(db_psql.String(100))
    username = db_psql.Column(db_psql.String(200))
    password = db_psql.Column(db_psql.String(200))
    credit = db_psql.Column(db_psql.Integer)

    def __repr__(self):
        return f'User Name: {self.username}\nEmail: {self.email}\
            \nAdmin: {bool(self.admin)}\nCredit: {bool(self.credit)}'

    def __init__(self, id, name, email, username, password, credit=0, admin=False):
        self.username = username
        self.email = email
        self.name = name
        self.password = password
        self.id = id
        self.admin = admin
        self.credit = credit


class questions_db(db_psql.Model):
    id = db_psql.Column(db_psql.String(200), primary_key=True)
    question = db_psql.Column(db_psql.Text)
    author = db_psql.Column(db_psql.String(
        200), db_psql.ForeignKey("user_db.id"))
    likes = db_psql.Column(db_psql.Integer)
    time_asked = db_psql.Column(db_psql.DateTime)

    def __repr__(self):
        return f'User ID: {self.author}\nQuestion: {self.question}\nPopularity: {self.likes}'

    def __init__(self, id, question, author, asked_on, likes=0):
        self.id = id
        self.question = question
        self.likes = likes
        self.author = author
        self.time_asked = asked_on


class answers_db(db_psql.Model):
    id = db_psql.Column(db_psql.String(200), primary_key=True)
    answers = db_psql.Column(db_psql.Text)
    author = db_psql.Column(db_psql.String(
        200), db_psql.ForeignKey('user_db.id'))
    question = db_psql.Column(db_psql.String(
        200), db_psql.ForeignKey('questions_db.id'))
    time_answered = db_psql.Column(db_psql.DateTime)

    def __repr__(self):
        return f'Question ID: {self.question}\nAnswer: {self.answers}'

    def __init__(self, id, answer, author, time_answer, question):
        self.id = id
        self.answers = answer
        self.author = author
        self.question = question
        self.time_answered = time_answer

# this is the question likes and dislikes table schema,
# the frontned sends 1 or 0 as the response which is then
# interpreter as like and dislike respectively


class question_responses(db_psql.Model):
    id = db_psql.Column(db_psql.String(200), primary_key=True)
    time = db_psql.Column(db_psql.DateTime)
    question_id = db_psql.Column(db_psql.String(
        200), db_psql.ForeignKey('questions_db.id'))
    user_id = db_psql.Column(db_psql.String(
        200), db_psql.ForeignKey('user_db.id'))
    response = db_psql.Column(db_psql.Boolean)

    def __repr__(self):
        if self.response == True:
            resp = 'Liked'
        elif self.response == False:
            resp = 'Disliked'
        else:
            resp = 'Unknown'
        return f'Question ID: {self.question_id} \
                is {resp} by {self.user_id} at {self.time}'

    def __init__(self, id, response, user, question, time):
        self.time = time
        self.id = id
        self.response = response
        self.user_id = user
        self.question_id = question


db_psql.create_all()


In [7]:
app_sql = Flask(__name__)
# "sqlite:///database2.sqlite3"
app_sql.config['SQLALCHEMY_DATABASE_URI'] = deployment[0]
app_sql.config['SECRET_KEY'] = secrets.token_hex(15)
app_sql.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True
db = SQLAlchemy(app_sql)


class user_db(db.Model):
    id = db.Column(db.String(200), primary_key=True)
    admin = db.Column(db.Boolean)
    name = db.Column(db.String(200))
    email = db.Column(db.String(100))
    username = db.Column(db.String(200))
    password = db.Column(db.String(200))
    credit = db.Column(db.Integer)

    def __repr__(self):
        return f'User Name: {self.username}\nEmail: {self.email}\
            \nAdmin: {bool(self.admin)}\nCredit: {bool(self.credit)}'

    def __init__(self, id, name, email, username, password, credit=0, admin=False):
        self.username = username
        self.email = email
        self.name = name
        self.password = password
        self.id = id
        self.admin = admin
        self.credit = credit


class questions_db(db.Model):
    id = db.Column(db.String(200), primary_key=True)
    question = db.Column(db.Text)
    author = db.Column(db.String(200), db.ForeignKey("user_db.id"))
    likes = db.Column(db.Integer)
    time_asked = db.Column(db.DateTime)

    def __repr__(self):
        return f'User ID: {self.author}\nQuestion: {self.question}\nPopularity: {self.likes}'

    def __init__(self, id, question, author, asked_on, likes=0):
        self.id = id
        self.question = question
        self.likes = likes
        self.author = author
        self.time_asked = asked_on


class answers_db(db.Model):
    id = db.Column(db.String(200), primary_key=True)
    answers = db.Column(db.Text)
    author = db.Column(db.String(200), db.ForeignKey('user_db.id'))
    question = db.Column(db.String(200), db.ForeignKey('questions_db.id'))
    time_answered = db.Column(db.DateTime)

    def __repr__(self):
        return f'Question ID: {self.question}\nAnswer: {self.answers}'

    def __init__(self, id, answer, author, time_answer, question):
        self.id = id
        self.answers = answer
        self.author = author
        self.question = question
        self.time_answered = time_answer

# this is the question likes and dislikes table schema,
# the frontned sends 1 or 0 as the response which is then
# interpreter as like and dislike respectively

# we can change class names without changing the table names, this is very hepful


class question_responses_old(db.Model):
    __tablename__ = "question_responses"
    time = db.Column(db.DateTime, primary_key=True)
    question_id = db.Column(db.String(
        200), db.ForeignKey('questions_db.id'))
    user_id = db.Column(db.String(
        200), db.ForeignKey('user_db.id'))
    response = db.Column(db.Boolean)

    def __repr__(self):
        if self.response == True:
            resp = 'Liked'
        elif self.response == False:
            resp = 'Disliked'
        else:
            resp = 'Unknown'
        return f'Question ID: {self.question_id} \
                is {resp} by {self.user_id} at {self.time}'

    def __init__(self, id, response, user, question, time):
        self.time = time
        self.id = id
        self.response = response
        self.user_id = user
        self.question_id = question


db.create_all()


In [4]:
query = db.session.query(user_db)
print(query)

for q in query:
    print(q)
    item = user_db(id=q.id, name=q.name, email=q.email, credit=q.credit,
                   password=q.password, admin=q.admin, username=q.username)
    db_psql.session.add(item)
db_psql.session.commit()


SELECT user_db.id AS user_db_id, user_db.admin AS user_db_admin, user_db.name AS user_db_name, user_db.email AS user_db_email, user_db.username AS user_db_username, user_db.password AS user_db_password, user_db.credit AS user_db_credit 
FROM user_db
User Name: Prabu
Email: prabu@wipro.com            
Admin: False
Credit: True
User Name: Sagnik
Email: sagnik@wipro.com            
Admin: False
Credit: True
User Name: Joshva
Email: joshva@wipro.com            
Admin: False
Credit: True
User Name: Shivaganesh
Email: shivaganesh@wipro.com            
Admin: False
Credit: True
User Name: Shruti
Email: shruti@wipro.com            
Admin: False
Credit: True
User Name: Bhavana
Email: bhavana@wipro.com            
Admin: False
Credit: True
User Name: Tirupathi
Email: tirupathi@wipro.com            
Admin: False
Credit: True
User Name: Neetha
Email: neetha@wipro.com            
Admin: False
Credit: True
User Name: Ashwin
Email: ashwin@wipro.com            
Admin: False
Credit: True
User Name: Siv

In [5]:
query = db.session.query(questions_db)
print(query)

for q in query:
    print(q)
    item = questions_db(id=q.id, question=q.question,
                        author=q.author, asked_on=q.time_asked, likes=q.likes)
    db_psql.session.add(item)
db_psql.session.commit()


SELECT questions_db.id AS questions_db_id, questions_db.question AS questions_db_question, questions_db.author AS questions_db_author, questions_db.likes AS questions_db_likes, questions_db.time_asked AS questions_db_time_asked 
FROM questions_db
User ID: 32ef1187-3506-4cc5-abec-8cbfdbe5970e
Question: How are streams allocated in internship?
Popularity: 4
User ID: 1506c809-2af6-433c-a06f-764d38d4a543
Question: What is internship?
Popularity: 5
User ID: 26d48cd3-e9b3-42dd-b467-fb7f1774c01b
Question: What is velocity?
Popularity: 4
User ID: 483d29e9-9134-4302-b7d5-ee86859bca7e
Question: Can I do Internship program along with Velocity Program?
Popularity: 1
User ID: 512dd3c3-c014-41e5-81bf-686f9a2d7754
Question: Can I do PJP and Internship?
Popularity: 2
User ID: 4745dc2b-dcd4-420f-b5de-b3525d0c163a
Question: In between the Internship program if I leave can I get the Internship certificate?
Popularity: 1
User ID: e3f9b12c-4635-421e-b7c5-c02562be4cef
Question: What is duration of the Inter

In [6]:
query = db.session.query(answers_db)
print(query)

for q in query:
    print(q)
    item = answers_db(id=q.id, answer=q.answers, author=q.author,
                      time_answer=q.time_answered, question=q.question)
    db_psql.session.add(item)
db_psql.session.commit()


SELECT answers_db.id AS answers_db_id, answers_db.answers AS answers_db_answers, answers_db.author AS answers_db_author, answers_db.question AS answers_db_question, answers_db.time_answered AS answers_db_time_answered 
FROM answers_db
Question ID: d190ab3d-ec57-4174-a1e7-f22babfd05bc
Answer: Streams are allocated based on the demand from the business
Question ID: 8b6c1a8d-5dd7-43bd-a6da-25287760c57e
Answer: This is a 10/16 week program from Wipro to fulfill the last semester requirements of your college
Question ID: 8b6c1a8d-5dd7-43bd-a6da-25287760c57e
Answer: In an internship you will work on the Learning, Exploration and Capstone Projects.
Question ID: 44c60a14-d1d4-42cf-af0b-9d8a2c13ce3d
Answer: Based on the business demands, Wipro had tied up with External Vendors to train people on the niche skills.

Question ID: 62fb7534-f2c5-4d7f-95e0-d57cae499886
Answer: No. You have to choose either Internship or Velocity.
Question ID: 8b6c1a8d-5dd7-43bd-a6da-25287760c57e
Answer: Post joining 

In [9]:
query = db.session.query(question_responses_old)
print(query)

for q in query:
    print(q)
    item = question_responses(id=str(uuid.uuid4()),
                              response=q.response, user=q.user_id, question=q.question_id, time=q.time)
    db_psql.session.add(item)
db_psql.session.commit()


SELECT question_responses.time AS question_responses_time, question_responses.question_id AS question_responses_question_id, question_responses.user_id AS question_responses_user_id, question_responses.response AS question_responses_response 
FROM question_responses
Question ID: d190ab3d-ec57-4174-a1e7-f22babfd05bc                 is Liked by 26d48cd3-e9b3-42dd-b467-fb7f1774c01b at 2022-05-04 00:51:55.101828
Question ID: d190ab3d-ec57-4174-a1e7-f22babfd05bc                 is Liked by 483d29e9-9134-4302-b7d5-ee86859bca7e at 2022-05-04 01:04:02.406465
Question ID: 8b6c1a8d-5dd7-43bd-a6da-25287760c57e                 is Liked by 483d29e9-9134-4302-b7d5-ee86859bca7e at 2022-05-04 01:04:03.587542
Question ID: d190ab3d-ec57-4174-a1e7-f22babfd05bc                 is Liked by 32ef1187-3506-4cc5-abec-8cbfdbe5970e at 2022-05-04 01:06:36.076331
Question ID: 44c60a14-d1d4-42cf-af0b-9d8a2c13ce3d                 is Liked by 32ef1187-3506-4cc5-abec-8cbfdbe5970e at 2022-05-04 01:06:39.138074
Question

In [10]:
query = db.session.query(question_responses_old)
print(query)
for q in query:
    print(q)

SELECT question_responses.time AS question_responses_time, question_responses.question_id AS question_responses_question_id, question_responses.user_id AS question_responses_user_id, question_responses.response AS question_responses_response 
FROM question_responses
Question ID: d190ab3d-ec57-4174-a1e7-f22babfd05bc                 is Liked by 26d48cd3-e9b3-42dd-b467-fb7f1774c01b at 2022-05-04 00:51:55.101828
Question ID: d190ab3d-ec57-4174-a1e7-f22babfd05bc                 is Liked by 483d29e9-9134-4302-b7d5-ee86859bca7e at 2022-05-04 01:04:02.406465
Question ID: 8b6c1a8d-5dd7-43bd-a6da-25287760c57e                 is Liked by 483d29e9-9134-4302-b7d5-ee86859bca7e at 2022-05-04 01:04:03.587542
Question ID: d190ab3d-ec57-4174-a1e7-f22babfd05bc                 is Liked by 32ef1187-3506-4cc5-abec-8cbfdbe5970e at 2022-05-04 01:06:36.076331
Question ID: 44c60a14-d1d4-42cf-af0b-9d8a2c13ce3d                 is Liked by 32ef1187-3506-4cc5-abec-8cbfdbe5970e at 2022-05-04 01:06:39.138074
Question

In [12]:
query = db_psql.session.query(question_responses)
print(query)
for q in query:
    print(q.id)

SELECT question_responses.id AS question_responses_id, question_responses.time AS question_responses_time, question_responses.question_id AS question_responses_question_id, question_responses.user_id AS question_responses_user_id, question_responses.response AS question_responses_response 
FROM question_responses
7e0d02ed-acfd-449b-9f11-61ee1ef02685
93f0a1c4-bc7d-4208-9e5e-e84215596cd5
907c5087-fcc9-442e-8276-b2a024300662
6f72a3ad-4355-4277-ae6c-7fbbb72525c9
aff82e8f-65e5-400f-8253-fe037b4b453f
2b8cccdd-13d7-42ec-bccc-9d39f37425fc
c845b027-a758-4e65-af58-59292c084bdf
41718756-2891-48d6-91bb-3e48e9be0619
5f795d90-702f-46bd-8021-0f2c4d91a50f
793fc438-ef1d-4d28-be95-d3622b351681
bc0b3dc7-3347-4394-9970-404c6505f1ca
dd389b29-9c0f-44b1-902a-2031b358339e
2661feb6-b47a-480b-b206-fa26e2b9b3ab
6cb66cc1-2063-4bb4-a10a-abc7192ea60a
0bb1746f-edaf-4da9-93cc-a2f892e16790
baeff4e2-a518-4f5b-8327-165b9fe2ce5d
a48d81f0-3289-4cf1-8d9a-8e0527c60c6b
5466b1f0-72f7-434e-9993-ee6b51eaf7ef
547de41b-338a-4d1d-

In [15]:
db_psql.session.close()
db.session.close()